In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize
import os.path
import string
from ollama import Client
import random
import random

In [92]:
df_adm = pd.read_csv(r"D:\UT_Austin\sem2\mimic-iii-clinical-database-demo-1.4\admissions_master.csv")
df_adm.columns=df_adm.columns.str.upper()
# convert to dates
df_adm.ADMITTIME = pd.to_datetime(df_adm.ADMITTIME, errors = 'coerce')
df_adm.DISCHTIME = pd.to_datetime(df_adm.DISCHTIME, errors = 'coerce')
df_adm.DEATHTIME = pd.to_datetime(df_adm.DEATHTIME, errors = 'coerce')
# sort by subject_ID and admission date
df_adm = df_adm.sort_values(['SUBJECT_ID','ADMITTIME'])
df_adm = df_adm.reset_index(drop = True)
df_adm.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaT,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,ASIAN,NaN,NaN,NEWBORN,0,1
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,2101-10-20 17:09:00.000,2101-10-20 19:24:00.000,HYPOTENSION,0,1
2,3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,2191-03-15 13:10:00.000,2191-03-16 01:10:00.000,"FEVER,DEHYDRATION,FAILURE TO THRIVE",0,1
3,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaT,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,BUDDHIST,NaN,ASIAN,NaN,NaN,NEWBORN,0,1
4,5,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,CHRONIC RENAL FAILURE/SDA,0,1


In [93]:
df_adm.groupby(['ADMISSION_TYPE']).size()


ADMISSION_TYPE
ELECTIVE      7706
EMERGENCY    42071
NEWBORN       7863
URGENT        1336
dtype: int64

In [94]:
# check to see if there are any missing dates
print('Number of missing date admissions:', df_adm.ADMITTIME.isnull().sum())
print('Number of missing date discharges:', df_adm.DISCHTIME.isnull().sum())

Number of missing date admissions: 0
Number of missing date discharges: 0


In [95]:
# verify that it did what we wanted
df_adm.loc[df_adm.SUBJECT_ID == 124,['SUBJECT_ID','ADMITTIME','ADMISSION_TYPE']]

,SUBJECT_ID,ADMITTIME,ADMISSION_TYPE
165,124,2160-06-24 21:25:00,EMERGENCY
166,124,2161-12-17 03:39:00,EMERGENCY
167,124,2165-05-21 21:02:00,ELECTIVE
168,124,2165-12-31 18:55:00,EMERGENCY


In [96]:
# add the next admission date and type for each subject using groupby
# you have to use groupby otherwise the dates will be from different subjects
df_adm['NEXT_ADMITTIME'] = df_adm.groupby('SUBJECT_ID').ADMITTIME.shift(-1)
# get the next admission type
df_adm['NEXT_ADMISSION_TYPE'] = df_adm.groupby('SUBJECT_ID').ADMISSION_TYPE.shift(-1)

In [97]:
# verify that it did what we wanted
df_adm.loc[df_adm.SUBJECT_ID == 124,['SUBJECT_ID','ADMITTIME','ADMISSION_TYPE','NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']]

,SUBJECT_ID,ADMITTIME,ADMISSION_TYPE,NEXT_ADMITTIME,NEXT_ADMISSION_TYPE
165,124,2160-06-24 21:25:00,EMERGENCY,2161-12-17 03:39:00,EMERGENCY
166,124,2161-12-17 03:39:00,EMERGENCY,2165-05-21 21:02:00,ELECTIVE
167,124,2165-05-21 21:02:00,ELECTIVE,2165-12-31 18:55:00,EMERGENCY
168,124,2165-12-31 18:55:00,EMERGENCY,NaT,NaN


In [98]:
# get rows where next admission is elective and replace with naT or nan
rows = df_adm.NEXT_ADMISSION_TYPE == 'ELECTIVE'
df_adm.loc[rows,'NEXT_ADMITTIME'] = pd.NaT
df_adm.loc[rows,'NEXT_ADMISSION_TYPE'] = np.NaN

In [99]:
# verify that it did what we wanted
df_adm.loc[df_adm.SUBJECT_ID == 124,['SUBJECT_ID','ADMITTIME','ADMISSION_TYPE','NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']]

,SUBJECT_ID,ADMITTIME,ADMISSION_TYPE,NEXT_ADMITTIME,NEXT_ADMISSION_TYPE
165,124,2160-06-24 21:25:00,EMERGENCY,2161-12-17 03:39:00,EMERGENCY
166,124,2161-12-17 03:39:00,EMERGENCY,NaT,NaN
167,124,2165-05-21 21:02:00,ELECTIVE,2165-12-31 18:55:00,EMERGENCY
168,124,2165-12-31 18:55:00,EMERGENCY,NaT,NaN


In [100]:
# sort by subject_ID and admission date
# it is safer to sort right before the fill incase something changed the order above
df_adm = df_adm.sort_values(['SUBJECT_ID','ADMITTIME'])

# back fill (this will take a little while)
df_adm[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = df_adm.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')

C:\Users\ajays\AppData\Local\Temp\ipykernel_415092\1783507809.py:6: FutureWarning: DataFrameGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use DataFrame.fillna instead
  df_adm[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = df_adm.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')
C:\Users\ajays\AppData\Local\Temp\ipykernel_415092\1783507809.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_adm[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = df_adm.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')
C:\Users\ajays\AppData\Local\Temp\ipykernel_415092\1783507809.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future versio

In [101]:
# verify that it did what we wanted
df_adm.loc[df_adm.SUBJECT_ID == 124,['SUBJECT_ID','ADMITTIME','ADMISSION_TYPE','NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']]

,SUBJECT_ID,ADMITTIME,ADMISSION_TYPE,NEXT_ADMITTIME,NEXT_ADMISSION_TYPE
165,124,2160-06-24 21:25:00,EMERGENCY,2161-12-17 03:39:00,EMERGENCY
166,124,2161-12-17 03:39:00,EMERGENCY,2165-12-31 18:55:00,EMERGENCY
167,124,2165-05-21 21:02:00,ELECTIVE,2165-12-31 18:55:00,EMERGENCY
168,124,2165-12-31 18:55:00,EMERGENCY,NaT,NaN


In [102]:
# calculate the number of days between discharge and next admission
df_adm['DAYS_NEXT_ADMIT']=  (df_adm.NEXT_ADMITTIME - df_adm.DISCHTIME).dt.total_seconds()/(24*60*60)

In [103]:

print('Number with a readmission:', (~df_adm.DAYS_NEXT_ADMIT.isnull()).sum())
print('Total Number:', len(df_adm))

Number with a readmission: 11399
Total Number: 58976


In [104]:
# verify that it did what we wanted
df_adm.loc[df_adm.SUBJECT_ID == 124,['SUBJECT_ID','ADMITTIME','ADMISSION_TYPE','DISCHTIME','NEXT_ADMITTIME','NEXT_ADMISSION_TYPE', 'DAYS_NEXT_ADMIT']]

,SUBJECT_ID,ADMITTIME,ADMISSION_TYPE,DISCHTIME,NEXT_ADMITTIME,NEXT_ADMISSION_TYPE,DAYS_NEXT_ADMIT
165,124,2160-06-24 21:25:00,EMERGENCY,2160-07-15 15:10:00,2161-12-17 03:39:00,EMERGENCY,519.520139
166,124,2161-12-17 03:39:00,EMERGENCY,2161-12-24 15:35:00,2165-12-31 18:55:00,EMERGENCY,1468.138889
167,124,2165-05-21 21:02:00,ELECTIVE,2165-06-06 16:00:00,2165-12-31 18:55:00,EMERGENCY,208.121528
168,124,2165-12-31 18:55:00,EMERGENCY,2166-02-01 06:55:00,NaT,NaN,NaN


In [105]:
df_notes = pd.read_csv(r"D:\UT_Austin\sem2\mimic-iii-clinical-database-demo-1.4\all_discharge_summary.csv")
# df_notes = pd.read_csv("diabetes_patients_discharge_summary.csv")
df_notes.columns=df_notes.columns.str.upper()
print('Number of notes:',len(df_notes))

Number of notes: 59652


In [106]:
df_notes.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,1227,3403,172500,2172-06-20,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2172-6-15**] ...
1,2055,17137,174999,2156-07-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2156-6-29**] Dischar...
2,2077,6541,110445,2133-04-02,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2133-3-28**] Dischar...
3,6427,31203,139699,2148-08-12,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2148-7-29**] ...
4,12491,13422,199876,2148-05-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2148-4-23**] Dischar...


In [107]:
df_notes.CATEGORY.unique()

array(['Discharge summary'], dtype=object)

In [108]:
# look at the first note
df_notes.TEXT.iloc[0]

'Admission Date:  [**2172-6-15**]              Discharge Date:   [**2172-6-20**]\n\nDate of Birth:  [**2099-5-18**]             Sex:   F\n\nService: CARDIOTHORACIC\n\nAllergies:\nPatient recorded as having No Known Allergies to Drugs\n\nAttending:[**First Name3 (LF) 1283**]\nChief Complaint:\nDyspnea on exertion\n\nMajor Surgical or Invasive Procedure:\nMitral Valve Replacement, Cryomaze procedure and Left Atrial\nAppendage Resection [**2172-6-16**]\n\nHistory of Present Illness:\n72 y/o female with known mitral regurgitation who was referred\nfor surgical intervention.\n\nPast Medical History:\nMitral Regurgitation, Atrial Fibrillation h/o cardioversion,\nHypertension, Chronic Obstructive Pulmonary Disease, Hodgkin\'s\ndisease s/p Chemo, Squamous Cell cncer right calf s/p excision,\nHypothyroidism, s/p Hysterectomy, s/p c-section and\nappendectomy, s/p lumbar disc surgery\n\nSocial History:\nDenies tobacco use. Occasional ETOH. Lives with husband.\n\nFamily History:\nNon-contributory\

In [109]:

# filter to discharge summary
df_notes_dis_sum = df_notes.loc[df_notes.CATEGORY == 'Discharge summary']

In [110]:
df_notes_dis_sum_last = (df_notes_dis_sum.groupby(['SUBJECT_ID','HADM_ID']).nth(-1)).reset_index()
assert df_notes_dis_sum_last.duplicated(['HADM_ID']).sum() == 0, 'Multiple discharge summaries per admission'

In [111]:

df_adm_notes = pd.merge(df_adm[['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','DAYS_NEXT_ADMIT','NEXT_ADMITTIME','ADMISSION_TYPE','DEATHTIME']],
                        df_notes_dis_sum_last[['SUBJECT_ID','HADM_ID','TEXT']],
                        on = ['SUBJECT_ID','HADM_ID'],
                        how = 'left')
assert len(df_adm) == len(df_adm_notes), 'Number of rows increased'

In [112]:

print('Fraction of missing notes:', df_adm_notes.TEXT.isnull().sum() / len(df_adm_notes))
print('Fraction notes with newlines:', df_adm_notes.TEXT.str.contains('\n').sum() / len(df_adm_notes))
print('Fraction notes with carriage returns:', df_adm_notes.TEXT.str.contains('\r').sum() / len(df_adm_notes))

Fraction of missing notes: 0.1059753119913185
Fraction notes with newlines: 0.8940246880086815
Fraction notes with carriage returns: 0.0
Fraction notes with carriage returns: 0.0


In [113]:
df_adm_notes.groupby('ADMISSION_TYPE').apply(lambda g: g.TEXT.isnull().sum())/df_adm_notes.groupby('ADMISSION_TYPE').size()

C:\Users\ajays\AppData\Local\Temp\ipykernel_415092\3060859928.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_adm_notes.groupby('ADMISSION_TYPE').apply(lambda g: g.TEXT.isnull().sum())/df_adm_notes.groupby('ADMISSION_TYPE').size()


ADMISSION_TYPE
ELECTIVE     0.048663
EMERGENCY    0.037983
NEWBORN      0.536691
URGENT       0.042665
dtype: float64

In [114]:

df_adm_notes_clean = df_adm_notes.loc[df_adm_notes.ADMISSION_TYPE != 'NEWBORN'].copy()

In [115]:
print('Fraction of missing notes:', df_adm_notes_clean.TEXT.isnull().sum() / len(df_adm_notes_clean))
print('Fraction notes with newlines:', df_adm_notes_clean.TEXT.str.contains('\n').sum() / len(df_adm_notes_clean))
print('Fraction notes with carriage returns:', df_adm_notes_clean.TEXT.str.contains('\r').sum() / len(df_adm_notes_clean))

Fraction of missing notes: 0.03971592354195606
Fraction notes with newlines: 0.9602840764580439
Fraction notes with carriage returns: 0.0


In [116]:
df_adm_notes_clean['OUTPUT_LABEL'] = (df_adm_notes_clean.DAYS_NEXT_ADMIT < 30).astype('int')
df_adm_notes_clean.iloc[0]

SUBJECT_ID                                                         3
HADM_ID                                                       145834
ADMITTIME                                        2101-10-20 19:08:00
DISCHTIME                                        2101-10-31 13:58:00
DAYS_NEXT_ADMIT                                                  NaN
NEXT_ADMITTIME                                                   NaT
ADMISSION_TYPE                                             EMERGENCY
DEATHTIME                                                        NaT
TEXT               Admission Date:  [**2101-10-20**]     Discharg...
OUTPUT_LABEL                                                       0
Name: 1, dtype: object

In [117]:

print('Number of positive samples:', (df_adm_notes_clean.OUTPUT_LABEL == 1).sum())
print('Number of negative samples:',  (df_adm_notes_clean.OUTPUT_LABEL == 0).sum())
print('Total samples:', len(df_adm_notes_clean))

Number of positive samples: 3004
Number of negative samples: 48109
Total samples: 51113


In [118]:

def preprocess_text(df):
    # This function preprocesses the text by filling not a number and replacing new lines ('\n') and carriage returns ('\r')
    df.TEXT = df.TEXT.fillna(' ')
    df.TEXT =df.TEXT.str.replace('\n',' ')
    df.TEXT =df.TEXT.str.replace('\r',' ')
    return df

In [119]:
df_adm_notes_clean = preprocess_text(df_adm_notes_clean)

In [120]:
df_adm_notes_clean.iloc[0].TEXT

"Admission Date:  [**2101-10-20**]     Discharge Date:  [**2101-10-31**]  Date of Birth:   [**2025-4-11**]     Sex:  M  Service:  Medicine  CHIEF COMPLAINT:  Admitted from rehabilitation for hypotension (systolic blood pressure to the 70s) and decreased urine output.  HISTORY OF PRESENT ILLNESS:  The patient is a 76-year-old male who had been hospitalized at the [**Hospital1 190**] from [**10-11**] through [**10-19**] of [**2101**] after undergoing a left femoral-AT bypass graft and was subsequently discharged to a rehabilitation facility.  On [**2101-10-20**], he presented again to the [**Hospital1 346**] after being found to have a systolic blood pressure in the 70s and no urine output for 17 hours. A Foley catheter placed at the rehabilitation facility yielded 100 cc of murky/brown urine.  There may also have been purulent discharge at the penile meatus at this time.  On presentation to the Emergency Department, the patient was without subjective complaints.  In the Emergency Depart

In [121]:
pos_df = df_adm_notes_clean[df_adm_notes_clean.OUTPUT_LABEL == 1]
neg_df = df_adm_notes_clean[df_adm_notes_clean.OUTPUT_LABEL == 0]
neg_df = df_adm_notes_clean[df_adm_notes_clean.ADMISSION_TYPE == 'ELECTIVE']

In [122]:
neg_df['ADMISSION_TYPE'].unique()

array(['ELECTIVE'], dtype=object)

In [123]:
neg_df.iloc[4]

SUBJECT_ID                                                        23
HADM_ID                                                       152223
ADMITTIME                                        2153-09-03 07:15:00
DISCHTIME                                        2153-09-08 19:10:00
DAYS_NEXT_ADMIT                                          1501.016667
NEXT_ADMITTIME                                   2157-10-18 19:34:00
ADMISSION_TYPE                                              ELECTIVE
DEATHTIME                                                        NaT
TEXT               Admission Date:  [**2153-9-3**]              D...
OUTPUT_LABEL                                                       0
Name: 21, dtype: object

In [124]:
data_string = pos_df.iloc[0].TEXT
data_string

'Admission Date:  [**2131-4-30**]              Discharge Date:   [**2131-5-8**]  Date of Birth:  [**2061-8-17**]             Sex:   M  Service: CARDIOTHORACIC  Allergies: Patient recorded as having No Known Allergies to Drugs  Attending:[**First Name3 (LF) 922**] Chief Complaint: positive stress test ; chest pain during hypertension to 220/110   Major Surgical or Invasive Procedure: Cardiac Catheterization [**2131-4-30**] CABGx5 (LIMA-LAD,SVG-Diag,SVG-OM1,SVG-OM2,SVG-PDA)[**5-4**]   History of Present Illness: Mr. [**Known lastname 27218**] is a 69 yo M w/ h/o HTN and significant tobacco use, who had a severe headache on [**2131-4-19**] without associated symptoms (including weakness, visual changes), and was found to be hypertensive to 220/110.  He was transferred via ambulence to [**Hospital 1474**] Hospital and in route complained of right sided chest discomfort, associated with dyspnea and diaphoreiss. By report he was administered SL NTG and nitro paste with improvement in BP and 

In [125]:
data_string = neg_df.iloc[2].TEXT
data_string

"Admission Date:  [**2134-12-27**]              Discharge Date:   [**2134-12-31**]  Date of Birth:  [**2087-7-14**]             Sex:   F  Service: CARDIOTHORACIC  Allergies: Bactrim / Ampicillin / Remeron  Attending:[**First Name3 (LF) 1283**] Chief Complaint: History of stroke  Major Surgical or Invasive Procedure: [**2134-12-27**] Minimally invasive closure of patent foramen ovale   History of Present Illness: Mrs. [**Known lastname 11679**] is a 47 year old female who suffered a cerebellar stroke in [**2134-3-9**]. Workup at that time revealed a patent foramen ovale. She is currently followed by Dr. [**Last Name (STitle) 1693**](neurologist) from the [**Hospital1 18**]. Full hypercoagulability workup was unremarkable. Since [**Month (only) 956**], she has had no other neurological events. In preperation for surgical intervention, she underwent cardiac catheterization in [**Month (only) **] which showed normal coronary arteries and normal left ventricular function.  Past Medical Hist

In [126]:
client = Client()
model_name ="gemma3:1b"
your_query = f"""
task: Will this patient be readmitted?
format: Yes/No
"""
# prompt=f"""
# Discharge Summary of Patient:
#  {pos_df.iloc[random.randint(0, len(pos_df) - 1)].TEXT}
#  Based on this data, answer the question: {your_query}
# """
prompt = f"""Discharge Summary of Patients:
input example1:  {neg_df.iloc[0].TEXT},
ouput example1:No,
input example2:  {neg_df.iloc[1].TEXT},
ouput example2: No ,
input example3:  {neg_df.iloc[2].TEXT},
ouput example3:No ,
input example4:  {neg_df.iloc[len(neg_df) - 3].TEXT},
ouput example4:No,
input example5:  {neg_df.iloc[len(neg_df) - 2].TEXT},
ouput example5: No ,
input example6:  {neg_df.iloc[len(neg_df) - 1].TEXT},
ouput example6:No ,
Based on this data, answer the question: 
discarge_summary:{neg_df.iloc[4].TEXT}
{your_query}, 
"""

In [127]:
try:
    response = client.chat(
        model=model_name,
        messages=[
            {
                'role': 'assistant',
                'content': prompt,
                
            },
        ]
    )
    print("\nOllama's Response:")
    print(response['message']['content'])

except Exception as e:
    print(f"Error communicating with Ollama: {e}")


Ollama's Response:
answer: No



In [128]:
def generate_prompt(strategy, query, examples=None, context=None, chain_of_thought=None):
    """
    Generate a prompt for the Ollama model based on the specified strategy.

    Parameters:
        strategy (str): The strategy to use ('zero-shot', 'one-shot', 'in-context', 'chain-of-thoughts').
        query (str): The query or task to be answered.
        examples (list of tuples): Examples for one-shot or in-context learning. Each tuple should contain (input, output).
        context (str): Additional context for in-context learning.
        chain_of_thought (str): Step-by-step reasoning for chain-of-thoughts.

    Returns:
        str: The generated prompt.
    """
    query= f"Generate and provide a mimic {strategy} prompt to know whether a patient will be readmitted or not.This prompt will be feed into AI answering agent. Here is the discarge summary {query}"
    try:
        response = client.chat(
            model='llama3.2:1b',
            messages=[
                {
                    'role': 'tool',
                    'content': query,
                    
                },
            ]
        )
        print("\nOllama's Response:")
        return response['message']['content']

    except Exception as e:
        print(f"Error communicating with Ollama: {e}")

In [129]:
print(pos_df.iloc[200].TEXT)

Admission Date:  [**2154-11-20**]       Discharge Date:  [**2154-11-29**]   Service:  [**Hospital1 **]  HISTORY OF PRESENT ILLNESS:   This is an 85-year-old male with multiple medical problems, who presents with a one day history of pleuritic back pain and shortness of breath.  It began the night prior to admission when he took a deep breath and noticed a pain between his scapulae.  It became progressively worse, to the point where he was unable to take deep breaths.  He had no recent cough, fever, chills, chest pain, headaches, nausea, vomiting, lightheadedness, weakness or dysuria.  His ambulation was limited by claudication, not dyspnea on exertion.  He denies baseline shortness of breath, dyspnea on exertion, paroxysmal nocturnal dyspnea, orthopnea, or lower extremity edema.  He had a recent right upper lobe pneumonia in [**Month (only) 205**], status post a seven day course of levofloxacin with an additional admission in [**Month (only) 216**], as his pneumonia did not clear.  At 

In [ ]:
prompt=generate_prompt('chain-of-thoughts',neg_df.iloc[200].TEXT)
print(prompt)



Ollama's Response:
I can provide a mimic few-shot prompt for the AI answering agent.

Here's a possible prompt:

"Generate a medical diagnosis and treatment plan for a patient with metastatic adenocarcinoma of unclear primary, likely lung cancer, who has developed pulmonary edema and congestive heart failure. The patient is 74 years old with a history of previous XRT for supraclavicular lymph node metastases, chronic obstructive pulmonary disease (COPD), and hypertension. The patient's condition was stabilized on multiple medications, including Digoxin, Coumadin, Lasix, Zantac, Lipitor, Valsartan, Hydrochlorothiazide, and Toprol XL."

This prompt provides the following information:

* The patient's medical history and current symptoms
* The patient's age, sex, and service information
* The diagnosis of metastatic adenocarcinoma of unclear primary, likely lung cancer
* The development of pulmonary edema and congestive heart failure
* The patient's medications and treatment plan

The pr

In [141]:
def generate_response(prompt):
    try:
        response = client.chat(
            model='gemma3:1b',
            messages=[
                {
                    'role': 'user',
                    'content': prompt,
                },
            ]
        )
        print("\nOllama's Response:")
        print(response['message']['content'])

    except Exception as e:
        print(f"Error communicating with Ollama: {e}")

In [142]:
your_query = f"""
task: Will this patient be readmitted?
format: Yes/No
"""
prompt=f"""
{prompt}
 Based on this data, answer the question: {your_query}
"""

In [143]:
generate_response(prompt)


Ollama's Response:
No



In [134]:
from spacy import displacy
import spacy
spacy.require_gpu()
nlp = spacy.load("en_ner_bc5cdr_md")
def visualize_entities(data):
    doc = []
    doc = nlp(data)  
    displacy.render(doc, style="ent", jupyter=True)
    print("****************************************************************************************************")

c:\Users\ajays\anaconda3\envs\py311_cuda_env\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_ner_bc5cdr_md' (0.4.0) was trained with spaCy v3.0.1 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [144]:
visualize_entities(prompt)

****************************************************************************************************
